[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/KI2022-tutorial-universal-differential-equations/main?filepath=04%20introduction%20to%20bayesian%20differential%20equations.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Introduction to bayesian differential equations in <img height="60px" style='height:60px;display:inline;' alt="Julia" src="https://julialang.org/assets/infra/logo.svg">

There are several well-known probabilistic programming packages for julia, all written in pure julia:
- [Turing.jl](https://github.com/TuringLang/Turing.jl) for bayesian inference
- [Gen.jl](https://github.com/probcomp/Gen.jl) with programmable inference
- [Soss.jl](https://github.com/cscherrer/Soss.jl)
- and many more

We use Turing, because it comes with the very good support for UDE.

In [ ]:
import Turing, Distributions, Random, Statistics, StatsBase, StatsPlots
import DifferentialEquations, Plots, LinearAlgebra
using CommonSolve: solve

# Turing.jl - Uncertainty Modelling via Bayesian Estimation

Let's get into Turing.jl via an example: The Coin Flip mini example.

find more details at https://turing.ml/dev/tutorials/00-introduction/

In [ ]:
# Set the true probability of heads in a coin.
p_true = 0.5

# Iterate from having seen 0 observations to 100 observations.
N = 100

# Draw data from a Bernoulli distribution, i.e. draw heads or tails.
Random.seed!(12)
data = Random.rand(Distributions.Bernoulli(p_true), N)

# Declare our Turing model.
Turing.@model function coinflip(y)
    # Our prior belief about the probability of heads in a coin.
    p ~ Distributions.Beta(1, 1)

    # The number of observations.
    yN = length(y)
    for n in 1:yN
        # Heads or tails of a coin are drawn from a Bernoulli distribution.
        y[n] ~ Distributions.Bernoulli(p)
    end
end

# Settings of the Hamiltonian Monte Carlo (HMC) sampler.
iterations = 1000
ϵ = 0.05
τ = 10

# Start sampling.
chain = StatsBase.sample(coinflip(data), Turing.HMC(ϵ, τ), iterations)
Plots.plot(chain)

In [ ]:
Plots.histogram(chain[:p])

**👉 It is your time:** Try different `N` above and see how our information about `p` improves/worsens

In [ ]:
# your space

# Bayesian Differential Equations

Let's assume noisy Lotka Volterra data

In [ ]:
function lotka_volterra(du, u, p, t)
    x, y = u
    α, β, δ, γ = p
    du[1] = dx = α*x - β*x*y
    du[2] = dy = -δ*y + γ*x*y
end
u0 = [1.0, 1.0]
tspan = (0.0, 10.0)
p = [1.5, 1.0, 3.0, 1.0]
ode_prob = DifferentialEquations.ODEProblem(lotka_volterra, u0, tspan, p)

In [ ]:
ode_sol = solve(ode_prob, saveat=0.1)
ode_data = (Array(ode_sol) + 0.8
           * Random.randn(size(Array(ode_sol))))
# Plot simulation & noisy observations
Plots.plot(ode_sol, alpha=0.3)
Plots.scatter!(ode_sol.t, ode_data', color=[1 2], label="")

Let's assume we only have predator-data (foxes)

In [ ]:
Turing.@model function fitlv(data::AbstractVector, ode_prob)
    # Prior distributions.
    α ~ Distributions.truncated(Distributions.Normal(1.5, 0.5), 0.5, 2.5)
    β ~ Distributions.truncated(Distributions.Normal(1.2, 0.5), 0, 2)
    γ ~ Distributions.truncated(Distributions.Normal(3.0, 0.5), 1, 4)
    δ ~ Distributions.truncated(Distributions.Normal(1.0, 0.5), 0, 2)
    p = [α, β, γ, δ]
    
    # Simulate Lotka-Volterra model but save only
    # the second state of the system (predators).
    predicted = solve(ode_prob, p=p, saveat=0.1, save_idxs=2)
    
    # Observations of the predators.
    σ ~ Distributions.InverseGamma(2, 3)
    data ~ Distributions.MvNormal(predicted.u, σ^2 * LinearAlgebra.I)
    return nothing
end

# fit model only to predators (foxes)
model = fitlv(ode_data[2, :], ode_prob)

Sample & plot (called data retroduction)

In [ ]:
# Sample 3 independent chains.
chain = StatsBase.sample(model, Turing.NUTS(0.45), Turing.MCMCSerial(), 5000, 3, progress=false)
posterior_samples = StatsBase.sample(chain[[:α, :β, :γ, :δ]], 300, replace=false)

Plots.plot(legend=false)
for p in eachrow(Array(posterior_samples))
    ode_sol_p = solve(ode_prob, p=p, saveat=0.1)
    Plots.plot!(ode_sol_p, alpha=0.1, color="#BBBBBB")
end

# Plot simulation and noisy observations.
Plots.plot!(ode_sol, color=[1 2], linewidth=1)
Plots.scatter!(ode_sol.t, ode_data', color=[1 2])

**👉 It is your time:** How can we check whether the MCMC Bayesian Estimation converged successfully?

In [ ]:
# your space

# That was the introduction to bayesian differential equations - Thank you for participating 🙂

If you have question, suggestions, or you are just interested in Julia, contact me:
- Stephan Sahm stephan.sahm@jolin.io

### Further Material

- [Tutorial Bayesian Differential Equations](https://turing.ml/dev/tutorials/10-bayesian-differential-equations/)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

#### Supported by [Jolin.io](https://www.jolin.io)

Jolin.io is an IT-consultancy for high-performance computing and data science

We are there to help you, if you want to
- try out Julia at your company, or
- transition Matlab, Fortran, R, Python, etc. to Julia
- or speed up your existing code